In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import os

A partir de la base de datos proporcionada, se realizará un programa que permita identificar las huellas del mismo usuario y descartar el resto, dando información cuantitativa de la similitud entre las huellas, pudiendo utilizar openCV, numpy, os, PyTorch,..., pero no funciones descargadas especialmente para trabajar con huellas dactilares.

Se obtendrá la función densidad de probabilidad, la curva DET, la curva ROC., el valor EER y el valor AUC.

Se realizará un póster (una única transparencia) y se presentará durante la última semana de clase. Además, se entregará el código usado.

En la presentación tienen que tratarse los siguientes puntos:

-Hipótesis de partida.

-Metodología utilizada y explicación de su desarrollo.

-Resultados (curvas, EER y AUC).

-Conclusiones.

-Fuentes principales consultadas.

In [3]:
#TODO:
# 1. Limpieza: id y separar la huella
# 2. Eliminación de ruido

#TODO: Data augmentation
# 1. Rotación: tener en cuenta el fondo
# luminosidad

#TODO: Moselo / sistema / lo q sea
# SIFT
# CONVOLUTIVA

# MATCHING DE PATRONES

## 1. Limpieza del dataset

### 1.2. Extraer la huella de la imagen

In [2]:
from clean_dataset.hand_cleaning import build_dataset
from clean_dataset.scikit_cleaning  import build_dataset_frangi

In [3]:
sample_path = r"\BaseDatosNISTsd09_pr"
output_hand_dataset = "./datasets/hand_dataset"
output_frangi_dataset = "./datasets/frangi_dataset"
sample_path = os.getcwd()+sample_path
subfolders= os.listdir(sample_path)

build_dataset_frangi(sample_path, subfolders, output_frangi_dataset)
build_dataset(sample_path, subfolders, output_hand_dataset)

## 2. Planteamiento del sistema

In [4]:
targets = {}
i = 0

for folder in subfolders:
    for file in os.listdir(sample_path + "/" + folder):
        targets[file] = 1
    if i == 6:
        break
    i += 1

i = 0
for folder in subfolders:
    if i > 6:
        for file in os.listdir(sample_path + "/" + folder):
            targets[file] = 0
    i += 1

### 2.1. SIFT (Scale-Invariant Feature Transform)

In [5]:
dataset_path = output_frangi_dataset
users = {}
i = 0

for subfolder in subfolders:
    path = os.listdir(dataset_path + '\\' + subfolder)
    route = dataset_path + '\\' + subfolder + '\\' + path[0]

    img = cv.imread(route)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    sift = cv.SIFT_create()
    kp, des = sift.detectAndCompute(gray, None)
    users[path[0]] = (kp, des, path[0])
    if i == 6:
        break
    i += 1

In [47]:
def find_match(new_kp, new_des, users, threshold=300):
    bf = cv.BFMatcher()
    for user_id, (kp, des, img) in users.items():
        matches = bf.knnMatch(new_des, des, k=2)
        sorted_matches = sorted(matches, key=lambda x: x[0].distance)

        if sorted_matches and sorted_matches[0][0].distance < threshold:
            print(sorted_matches[0][0].distance)
            return True, img

    return False, None 

In [48]:
correct = 0
total = 0
for subfolder in subfolders:
    for file in os.listdir(dataset_path + '\\' + subfolder):
        path = dataset_path + '\\' + subfolder + '\\' + file
        true_target = targets[file]
        img = cv.imread(path)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        sift = cv.SIFT_create()
        kp, des = sift.detectAndCompute(gray, None)
        sift = cv.SIFT_create()
        new_kp, new_des = sift.detectAndCompute(gray, None)

        match_found, img_name = find_match(new_kp, new_des, users)
        if int(match_found) == true_target:
            correct += 1
        total += 1

0.0
242.8271026611328
250.76683044433594
250.72494506835938
245.53817749023438
253.70455932617188
248.0967559814453
254.1731719970703
241.49327087402344
262.4919128417969
253.7301788330078
245.0816192626953
225.7011260986328
258.2673034667969
247.2286376953125
237.86761474609375
249.5996856689453
239.81451416015625
243.0061798095703
247.0647735595703


In [49]:
print("Correct: ", correct)
print("Total: ", total)
print("Accuracy: ", correct / total * 100, "%")

Correct:  14
Total:  20
Accuracy:  70.0 %


### 2.2. ResNet pre-entrenada

In [104]:
import cv2 as cv
import torchvision.models as models
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
import numpy as np
import os
from scipy.spatial.distance import cosine, euclidean

In [123]:
def extract_features(img_path, model):
    model.eval()
    image = cv.imread(img_path)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image).unsqueeze(0)

    with torch.no_grad():
        features = model(image).squeeze().numpy()
    return features

def compare_fingerprints(users, v2, threshold=0.12):
    for idx, user in enumerate(users):
        user = user.flatten() if user.ndim > 1 else user
        v2 = v2.flatten() if v2.ndim > 1 else v2
        
        distance = cosine(user, v2)
        print(f"Distance with user {idx}: {distance}")
        if distance < threshold:
            print(f"Match found with user {distance} at index {idx}")
            print("-------------------------------------------------------")
            return idx
    return None

In [124]:
model = models.resnet50(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])

In [125]:
users = []
sample_path = r"\BaseDatosNISTsd09_pr"
dataset_path = "./Datasets"
sample_path = os.getcwd() + sample_path
subfolders = os.listdir(sample_path)

for i, subfolder in enumerate(subfolders[:6]):
    path = os.listdir(dataset_path + '\\' + subfolder)
    route = dataset_path + '\\' + subfolder + '\\' + path[0]
    vector = extract_features(route, model)
    users.append(vector)

In [126]:
matches = 0
total_checks = 0

for subfolder in subfolders:
    path = os.listdir(dataset_path + '\\' + subfolder)
    
    for file_name in path:
        route = dataset_path + '\\' + subfolder + '\\' + file_name
        v2 = extract_features(route, model)
        
        result = compare_fingerprints(users, v2)
        
        if result is not None:
            #print(f"Match found for folder '{subfolder}' with index {result}")
            matches += 1
        else:
            print(f"No match found for folder '{subfolder}'")
        
        total_checks += 1

accuracy = matches / total_checks if total_checks > 0 else 0
print(f"Accuracy: {accuracy * 100:.2f}% ({matches}/{total_checks})")

Distance with user 0: 0.0
Match found with user 0.0 at index 0
-------------------------------------------------------
Distance with user 0: 0.08758842945098877
Match found with user 0.08758842945098877 at index 0
-------------------------------------------------------
Distance with user 0: 0.14687985181808472
Distance with user 1: 0.0
Match found with user 0.0 at index 1
-------------------------------------------------------
Distance with user 0: 0.08757197856903076
Match found with user 0.08757197856903076 at index 0
-------------------------------------------------------
Distance with user 0: 0.11098814010620117
Match found with user 0.11098814010620117 at index 0
-------------------------------------------------------
Distance with user 0: 0.11199730634689331
Match found with user 0.11199730634689331 at index 0
-------------------------------------------------------
Distance with user 0: 0.13941192626953125
Distance with user 1: 0.10942977666854858
Match found with user 0.10942977

In [78]:
for subfolder in subfolders:
    files = os.listdir(dataset_path + '\\' + subfolder)

    for route_file in files:
        route = dataset_path + '\\' + subfolder + '\\' + route_file

        v2 = extract_features(route, model)
        distance = compare_fingerprints(users, v2)

In [54]:
vgg16 = models.vgg16(pretrained=True)
print(vgg16)
vgg16_latent = nn.Sequential(*list(vgg16.children())[:-1])
print(vgg16_latent)

c:\Users\Susana\anaconda3\envs\piav\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Susana\anaconda3\envs\piav\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [52]:
vgg16_latent.eval()
vectors = {}
sample_path = r"\BaseDatosNISTsd09_pr"
dataset_path = "./Datasets"
dataset_path = "./frangi_dataset"
sample_path = os.getcwd() + sample_path
subfolders = os.listdir(sample_path)
i = 0

for subfolder in subfolders:

    path = os.listdir(dataset_path + '\\' + subfolder)
    route = dataset_path + '\\' + subfolder + '\\' + path[0]

    img = cv.imread(route)
    img = cv.resize(img, (224, 224))
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float()

    with torch.no_grad():
        vector = vgg16_latent(img)

    vectors[path[0]] = vector

    if i == 6:
        break
    i += 1
    

In [53]:
print(vectors)

{'crd_0811f_01.png': tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000, 15.5075,  5.5852,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  2.6924,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         